In [2]:
import pandas as pd

In [3]:
train = pd.read_csv("/hwj/ant/atec_anti_fraud_train.csv")
test_a = pd.read_csv("/hwj/ant/atec_anti_fraud_test_a.csv")
test_b = pd.read_csv("/hwj/ant/atec_anti_fraud_test_b.csv")

In [4]:
train.loc[train['label'] == -1,'label']=1
train.fillna(0)
test_b.fillna(0)
validation = train[train["date"] > 20171021]
train = train[train["date"] <= 20171021]

In [6]:
train.drop(['id', 'date'], axis=1, inplace=True)
validation.drop(['id', 'date'], axis=1, inplace=True)
train = train.reset_index(drop=True)
validation = validation.reset_index(drop=True)

ValueError: labels ['id' 'date'] not contained in axis

In [7]:
train_Y = train.pop('label').values
train_X = train
validation_Y = validation.pop('label').values
validation_X = validation

In [8]:
import numpy as np
train_val_features = np.concatenate((train_X, validation_X),axis=0)
train_val_labels = np.concatenate((train_Y, validation_Y),axis=0)

In [9]:
import pandas as pd
import time
import datetime
from sklearn.utils import shuffle
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional     scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search
from sklearn.cross_validation import PredefinedSplit

/Users/hwj/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/hwj/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [10]:
test_fold = np.zeros(train_val_features.shape[0])   # 将所有index初始化为0,0表示第一轮的验证集
test_fold[:train_X.shape[0]] = -1            # 将训练集对应的index设为-1，表示永远不划分到验证集中
ps = PredefinedSplit(test_fold=test_fold)

In [11]:
param_test1 = {
 'max_depth':list(range(3,10,2)),
 'min_child_weight':list(range(1,6,2))
}

In [11]:
xgb1 = XGBClassifier(learning_rate = 0.1, 
                     n_estimators = 140, 
                     gamma = 0, 
                     subsample = 0.8, 
                     colsample_bytree = 0.8, 
                     objective = 'binary:logistic', 
                     scale_pos_weight = 1, 
                     seed = 27)

In [12]:
gsearch1 = GridSearchCV(estimator = xgb1, 
                        param_grid = param_test1, 
                        scoring='roc_auc', 
                        n_jobs=4, 
                        iid=False, 
                        cv=ps)

In [13]:
gsearch1.fit(train_val_features, train_val_labels)

GridSearchCV(cv=sklearn.cross_validation.PredefinedSplit(test_fold=[-1 -1 ...,  0  0]),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=140,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'max_depth': [3, 5, 7, 9], 'min_child_weight': [1, 3, 5]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [17]:
dir(gsearch1)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__getstate__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_estimator_type',
 '_fit',
 '_get_param_names',
 'best_estimator_',
 'best_params_',
 'best_score_',
 'cv',
 'decision_function',
 'error_score',
 'estimator',
 'fit',
 'fit_params',
 'get_params',
 'grid_scores_',
 'iid',
 'inverse_transform',
 'n_jobs',
 'param_grid',
 'pre_dispatch',
 'predict',
 'predict_log_proba',
 'predict_proba',
 'refit',
 'score',
 'scorer_',
 'scoring',
 'set_params',
 'transform',
 'verbose']

In [20]:
gsearch1.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=9, min_child_weight=5, missing=None, n_estimators=140,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8)

In [21]:
gsearch1.best_params_

{'max_depth': 9, 'min_child_weight': 5}

In [22]:
test_b.head()

,id,date,f1,f2,f3,f4,f5,f6,f7,f8,...,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
0,661ae9a03c6c74feab2555c9987e140ae3f5421bd8e7a0...,20180305,0,0,1,1,100810.0,0,4,2,...,301.0,301.0,302.0,192.0,301.0,218.0,219.0,219.0,153.0,218.0
1,ada98d4358e72a27cc5e92f691a87a4fd62a7cda0387e2...,20180303,0,0,1,1,100810.0,1,6,1,...,7.0,23.0,25.0,5.0,6.0,2.0,1.0,1.0,2.0,2.0
2,e33675a962e5bf44d05a2b01903a4beb88a0c6385c05c6...,20180226,0,0,1,1,100811.0,1,2,1,...,301.0,302.0,302.0,130.0,292.0,225.0,225.0,225.0,104.0,218.0
3,4c5da176de7172dbd1ad0aa7edf9866548ec720d6c7318...,20180207,1,0,1,0,100811.0,0,5,1,...,301.0,302.0,301.0,256.0,302.0,262.0,261.0,261.0,221.0,261.0
4,fa39e3491900d49b862d30b5dbcd0b1c30bb4ff0d96396...,20180226,0,0,1,1,100811.0,1,7,1,...,301.0,302.0,301.0,108.0,301.0,210.0,209.0,209.0,80.0,210.0


In [24]:
test_ids = test_b.pop('id')

In [25]:
test_b.drop(['date'], axis=1, inplace=True)

In [26]:
test_b.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
0,0,0,1,1,100810.0,0,4,2,0,1,...,301.0,301.0,302.0,192.0,301.0,218.0,219.0,219.0,153.0,218.0
1,0,0,1,1,100810.0,1,6,1,0,2,...,7.0,23.0,25.0,5.0,6.0,2.0,1.0,1.0,2.0,2.0
2,0,0,1,1,100811.0,1,2,1,0,1,...,301.0,302.0,302.0,130.0,292.0,225.0,225.0,225.0,104.0,218.0
3,1,0,1,0,100811.0,0,5,1,2,1,...,301.0,302.0,301.0,256.0,302.0,262.0,261.0,261.0,221.0,261.0
4,0,0,1,1,100811.0,1,7,1,1,2,...,301.0,302.0,301.0,108.0,301.0,210.0,209.0,209.0,80.0,210.0


In [28]:
test_X = test_b
test_b_pred = gsearch1.predict_proba(test_X)

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', '

In [33]:
test_X.columns

Index([u'f1', u'f2', u'f3', u'f4', u'f5', u'f6', u'f7', u'f8', u'f9', u'f10',
       ...
       u'f288', u'f289', u'f290', u'f291', u'f292', u'f293', u'f294', u'f295',
       u'f296', u'f297'],
      dtype='object', length=297)

In [34]:
test_b_0 = pd.read_csv("/root/ant/atec_anti_fraud_test_b.csv")

In [35]:
test_b_0.columns

Index([u'id', u'date', u'f1', u'f2', u'f3', u'f4', u'f5', u'f6', u'f7', u'f8',
       ...
       u'f288', u'f289', u'f290', u'f291', u'f292', u'f293', u'f294', u'f295',
       u'f296', u'f297'],
      dtype='object', length=299)

In [36]:
train.columns

Index([u'f1', u'f2', u'f3', u'f4', u'f5', u'f6', u'f7', u'f8', u'f9', u'f10',
       ...
       u'f288', u'f289', u'f290', u'f291', u'f292', u'f293', u'f294', u'f295',
       u'f296', u'f297'],
      dtype='object', length=297)

In [101]:
test_b_0.drop(['date', 'id'], axis=1, inplace=True)

In [38]:
dir(train_val_features)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_finalize__',
 '__array_interface__',
 '__array_prepare__',
 '__array_priority__',
 '__array_struct__',
 '__array_wrap__',
 '__class__',
 '__complex__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__delslice__',
 '__div__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getslice__',
 '__gt__',
 '__hash__',
 '__hex__',
 '__iadd__',
 '__iand__',
 '__idiv__',
 '__ifloordiv__',
 '__ilshift__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lshift__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__oct__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__rdivmod__',


In [40]:
train_val_features

array([[   0.,    1.,    1., ...,  161.,    8.,  160.],
       [   1.,    1.,    0., ...,  182.,   51.,  181.],
       [   0.,    0.,    1., ...,    2.,    2.,    2.],
       ..., 
       [   0.,    1.,    1., ...,    2.,    2.,    2.],
       [   0.,    1.,    0., ...,  184.,   36.,  111.],
       [   0.,    1.,    0., ...,  189.,   93.,  189.]])

In [41]:
train.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
0,0,1,1,1,100805.0,1,5,2,1,1,...,302.0,324.0,391.0,13.0,302.0,160.0,160.0,161.0,8.0,160.0
1,1,1,0,1,100805.0,1,5,1,2,0,...,302.0,301.0,301.0,74.0,302.0,182.0,181.0,182.0,51.0,181.0
2,0,0,1,1,100802.0,0,6,2,1,2,...,6.0,6.0,5.0,5.0,6.0,2.0,1.0,2.0,2.0,2.0
3,0,1,1,0,100805.0,0,6,2,1,1,...,301.0,301.0,301.0,150.0,301.0,239.0,239.0,239.0,116.0,240.0
4,1,1,1,1,100802.0,0,7,2,1,1,...,2.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0


In [42]:
test_X.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
0,0,0,1,1,100810.0,0,4,2,0,1,...,301.0,301.0,302.0,192.0,301.0,218.0,219.0,219.0,153.0,218.0
1,0,0,1,1,100810.0,1,6,1,0,2,...,7.0,23.0,25.0,5.0,6.0,2.0,1.0,1.0,2.0,2.0
2,0,0,1,1,100811.0,1,2,1,0,1,...,301.0,302.0,302.0,130.0,292.0,225.0,225.0,225.0,104.0,218.0
3,1,0,1,0,100811.0,0,5,1,2,1,...,301.0,302.0,301.0,256.0,302.0,262.0,261.0,261.0,221.0,261.0
4,0,0,1,1,100811.0,1,7,1,1,2,...,301.0,302.0,301.0,108.0,301.0,210.0,209.0,209.0,80.0,210.0


In [43]:
test_X.columns = ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', 'f134', 'f135', 'f136', 'f137', 'f138', 'f139', 'f140', 'f141', 'f142', 'f143', 'f144', 'f145', 'f146', 'f147', 'f148', 'f149', 'f150', 'f151', 'f152', 'f153', 'f154', 'f155', 'f156', 'f157', 'f158', 'f159', 'f160', 'f161', 'f162', 'f163', 'f164', 'f165', 'f166', 'f167', 'f168', 'f169', 'f170', 'f171', 'f172', 'f173', 'f174', 'f175', 'f176', 'f177', 'f178', 'f179', 'f180', 'f181', 'f182', 'f183', 'f184', 'f185', 'f186', 'f187', 'f188', 'f189', 'f190', 'f191', 'f192', 'f193', 'f194', 'f195', 'f196', 'f197', 'f198', 'f199', 'f200', 'f201', 'f202', 'f203', 'f204', 'f205', 'f206', 'f207', 'f208', 'f209', 'f210', 'f211', 'f212', 'f213', 'f214', 'f215', 'f216', 'f217', 'f218', 'f219', 'f220', 'f221', 'f222', 'f223', 'f224', 'f225', 'f226', 'f227', 'f228', 'f229', 'f230', 'f231', 'f232', 'f233', 'f234', 'f235', 'f236', 'f237', 'f238', 'f239', 'f240', 'f241', 'f242', 'f243', 'f244', 'f245', 'f246', 'f247', 'f248', 'f249', 'f250', 'f251', 'f252', 'f253', 'f254', 'f255', 'f256', 'f257', 'f258', 'f259', 'f260', 'f261', 'f262', 'f263', 'f264', 'f265', 'f266', 'f267', 'f268', 'f269', 'f270', 'f271', 'f272', 'f273', 'f274', 'f275', 'f276', 'f277', 'f278', 'f279', 'f280', 'f281', 'f282', 'f283', 'f284', 'f285', 'f286', 'f287', 'f288', 'f289', 'f290', 'f291', 'f292', 'f293', 'f294', 'f295', 'f296']

In [44]:
test_X.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f287,f288,f289,f290,f291,f292,f293,f294,f295,f296
0,0,0,1,1,100810.0,0,4,2,0,1,...,301.0,301.0,302.0,192.0,301.0,218.0,219.0,219.0,153.0,218.0
1,0,0,1,1,100810.0,1,6,1,0,2,...,7.0,23.0,25.0,5.0,6.0,2.0,1.0,1.0,2.0,2.0
2,0,0,1,1,100811.0,1,2,1,0,1,...,301.0,302.0,302.0,130.0,292.0,225.0,225.0,225.0,104.0,218.0
3,1,0,1,0,100811.0,0,5,1,2,1,...,301.0,302.0,301.0,256.0,302.0,262.0,261.0,261.0,221.0,261.0
4,0,0,1,1,100811.0,1,7,1,1,2,...,301.0,302.0,301.0,108.0,301.0,210.0,209.0,209.0,80.0,210.0


In [45]:
test_b_pred = gsearch1.predict_proba(test_X)

In [46]:
test_b_pred

array([[  9.99790430e-01,   2.09586520e-04],
       [  9.99908447e-01,   9.15603305e-05],
       [  9.99857962e-01,   1.42063058e-04],
       ..., 
       [  9.98383939e-01,   1.61604746e-03],
       [  8.88688087e-01,   1.11311935e-01],
       [  9.94143367e-01,   5.85663924e-03]], dtype=float32)

In [48]:
rst = pd.DataFrame()
rst['id'] = test_ids
rst['score'] = test_b_pred[:,1]

In [49]:
rst.head()

,id,score
0,661ae9a03c6c74feab2555c9987e140ae3f5421bd8e7a0...,0.000210
1,ada98d4358e72a27cc5e92f691a87a4fd62a7cda0387e2...,0.000092
2,e33675a962e5bf44d05a2b01903a4beb88a0c6385c05c6...,0.000142
3,4c5da176de7172dbd1ad0aa7edf9866548ec720d6c7318...,0.054516
4,fa39e3491900d49b862d30b5dbcd0b1c30bb4ff0d96396...,0.002178


In [50]:
rst[['id', 'score']].to_csv('/root/ant/xgb_ant_b_0.csv', index=False, sep=',')